In [1]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999;

<IPython.core.display.Javascript object>

In [2]:
from ipywidgets import interact
import numpy as np

from bokeh.io import push_notebook, show, output_notebook, save
from bokeh.plotting import figure
from bokeh.charts import Line, Bar
from bokeh.models import ColumnDataSource, Range1d, LabelSet, Label
output_notebook()

Loading BokehJS ...

In [33]:
# Total loan amount you will pay
loan_amount = 195000
# Percentage you will put down(as a decimal 5% == 0.05)
down_payment_pct = .05
# Expected number of months you will live in house
expected_length = 4*12
# Loan Length(in years)
loan_length = 15
# The rates you want to look at(Base comparison rate needs to be first)
#rates = [0.0250, 0.0275, 0.02625, 0.02875, 0.03, 0.03125]
#rates = [0.0250, 0.03]
rates = [.02375, .0249, .025, .02625, .0275, .02875, .0299]#, .03, .03125, .0325, .03375, .0349, .035, .03625, .0375, .03875]
#rates = [.02375, 0.0275, 0.02875]
# Points costs for rates(must be same size as rates. negative numbers mean you are paid that amount for the loan)
#loan_costs = [2375, 0, 1187.5, -1781.25, -2968.75, -4156.25]
#loan_costs = [2375, -2968.75]
loan_costs = [4888.7475, 3932.8575, 3843.9375, 3084.4125, 774.345, -125.97, -905.8725]#, -979.9725, -1678.365, -2395.2825, -2991.7875, -3723.525, -3793.92, -4457.115, -5055.4725, -5550.09]
#loan_costs = [4888.7475, 774.345, -125.97]
assert len(rates) == len(loan_costs), 'Make sure rates and loan_costs have same length'
# Rates and costs together
rates_and_costs = zip(rates, loan_costs)
# The principal of the loan to be paid
principal_amount = loan_amount * (1-down_payment_pct)

print "Principal Amount: {0}".format(principal_amount)

Principal Amount: 185250.0


# Interest and Payment portions of payment

In [34]:
def compare(rate1, cost1, rate2, cost2, filterval=5):
    '''
    rate1 - percentage rate(eg. 0.0275)
    cost1 - cost of rate1(negative means money is given back to you)
    rate2 - percentage rate to compare with(eg. 0.025)
    cost2 - cost of rate2
    filtervalue - do not graph if break even months / 12 is <= than this
    '''
    rates_and_costs = [(rate1, cost1), (rate2, cost2)]
    # Financial start the period count at 1 so that is why + 1
    per = np.arange(12*loan_length) + 1
    cumulative_interest_paid = cip = {}
    monthly_payments = mp = []
    for rate, cost in rates_and_costs:
        # Interest payments over course of loan(every month)
        ipmts = -1 * np.ipmt(rate/12, per, 12*loan_length, principal_amount)
        # Principal payments over course of loan(every month)
        ppmts = -1 * np.ppmt(rate/12, per, 12*loan_length, principal_amount)
        # Calculates monthly payments of loan(every month)
        pmts = ipmts + ppmts
        # Cumulitive interest paid for each month
        cip[str(rate)] = np.cumsum(ipmts) + cost
        lines = {
            'Interest Pmt': ipmts,
            'Principal': ppmts,
            'Payment': pmts
        }
        # Holds monthly payments
        mp.append(pmts[0])
        #p = Line(lines, title='Rate: {0}'.format(rate))
        #show(p)
    cost_diff = cost1 - cost2
    mp_diff = np.absolute(mp[0] - mp[1])
    break_even_mp = cost_diff / mp_diff
    title='{0} vs {1}'.format(rate1, rate2)
    p = figure(title=title)
    colors = ['red', 'green', 'blue', 'orange']
    color_map = dict(zip(cip.keys(), colors))
    for k,v in cip.iteritems():
        p.line(x=range(len(v)), y=v, legend=str(k), line_color=color_map[k])
    vals = np.absolute(np.subtract(*cip.values()))
    break_even_months = vals.argmin()
    if filterval <= break_even_months/12.0:
        return
    print "{0}(Cost Rate1) - {1}(Cost Rate2) = {2}(Cost Diff)".format(cost1, cost2, cost_diff)
    print "{0}(Montly Payment1) - {1}(Monthly Payment2) = {2}(Payment Diff)".format(mp[0], mp[1], mp_diff)
    print "{0}(Cost Diff) / {1}(Payment Diff) = {2}(Break Even Point)".format(cost_diff, mp_diff, break_even_mp)
    break_even_value = cip.values()[0][break_even_months]
    break_even_point = Label(x=break_even_months, y=break_even_value, text='{0}m: {1}y'.format(break_even_months, break_even_months/12.0))
    break_even_mp_point = Label(x=break_even_mp, y=2, text='{0}m: {1}y'.format(break_even_mp, break_even_mp/12.0))
    p.add_layout(break_even_point)
    p.add_layout(break_even_mp_point)
    #p.add_layout(labels)
    show(p)

compare_years = 8
for i in range(len(rates)):
    rate = rates[i]
    cost = loan_costs[i]
    print rate, cost
    for r,c in rates_and_costs[i+1:]:
        print r, c
        compare(rate, cost, r, c, compare_years)

0.02375 4888.7475
0.0249 3932.8575
4888.7475(Cost Rate1) - 3932.8575(Cost Rate2) = 955.89(Cost Diff)
1224.35608985(Montly Payment1) - 1234.35515353(Monthly Payment2) = 9.99906367607(Payment Diff)
955.89(Cost Diff) / 9.99906367607(Payment Diff) = 95.5979510649(Break Even Point)


0.025 3843.9375
4888.7475(Cost Rate1) - 3843.9375(Cost Rate2) = 1044.81(Cost Diff)
1224.35608985(Montly Payment1) - 1235.22700969(Monthly Payment2) = 10.8709198401(Payment Diff)
1044.81(Cost Diff) / 10.8709198401(Payment Diff) = 96.1105421962(Break Even Point)


0.02625 3084.4125
4888.7475(Cost Rate1) - 3084.4125(Cost Rate2) = 1804.335(Cost Diff)
1224.35608985(Montly Payment1) - 1246.15720555(Monthly Payment2) = 21.8011156992(Payment Diff)
1804.335(Cost Diff) / 21.8011156992(Payment Diff) = 82.7634248125(Break Even Point)


0.0275 774.345
4888.7475(Cost Rate1) - 774.345(Cost Rate2) = 4114.4025(Cost Diff)
1224.35608985(Montly Payment1) - 1257.14658333(Monthly Payment2) = 32.7904934827(Payment Diff)
4114.4025(Cost Diff) / 32.7904934827(Payment Diff) = 125.475467521(Break Even Point)


0.02875 -125.97
4888.7475(Cost Rate1) - -125.97(Cost Rate2) = 5014.7175(Cost Diff)
1224.35608985(Montly Payment1) - 1268.195045(Monthly Payment2) = 43.8389551552(Payment Diff)
5014.7175(Cost Diff) / 43.8389551552(Payment Diff) = 114.38953055(Break Even Point)


...

0.02625 3084.4125
0.0275 774.345
0.02875 -125.97
0.0299 -905.8725
3084.4125(Cost Rate1) - -905.8725(Cost Rate2) = 3990.285(Cost Diff)
1246.15720555(Montly Payment1) - 1278.41172506(Monthly Payment2) = 32.2545195107(Payment Diff)
3990.285(Cost Diff) / 32.2545195107(Payment Diff) = 123.712430398(Break Even Point)


0.0275 774.345
0.02875 -125.97
774.345(Cost Rate1) - -125.97(Cost Rate2) = 900.315(Cost Diff)
1257.14658333(Montly Payment1) - 1268.195045(Monthly Payment2) = 11.0484616725(Payment Diff)
900.315(Cost Diff) / 11.0484616725(Payment Diff) = 81.4878149273(Break Even Point)


0.0299 -905.8725
774.345(Cost Rate1) - -905.8725(Cost Rate2) = 1680.2175(Cost Diff)
1257.14658333(Montly Payment1) - 1278.41172506(Monthly Payment2) = 21.2651417272(Payment Diff)
1680.2175(Cost Diff) / 21.2651417272(Payment Diff) = 79.0127581349(Break Even Point)


0.02875 -125.97
0.0299 -905.8725
-125.97(Cost Rate1) - -905.8725(Cost Rate2) = 779.9025(Cost Diff)
1268.195045(Montly Payment1) - 1278.41172506(Monthly Payment2) = 10.2166800547(Payment Diff)
779.9025(Cost Diff) / 10.2166800547(Payment Diff) = 76.3361968684(Break Even Point)


0.0299 -905.8725


In [ ]:
monthly_payments = np.pmt(np.array(rates)/12, 12*loan_length, principal_amount)
print "Monthly Payments: {0}".format(-1*monthly_payments)

In [ ]:
monthly_totals = [
    np.arange(loan_cost, -1*monthly_payment * (loan_length*12), -1*monthly_payment) 
    for rate, loan_cost, monthly_payment in zip(rates, loan_costs, monthly_payments)
]
lengths = [len(x) for x in monthly_totals]
max_length = max(lengths)
for i in range(len(monthly_totals)):
    x = monthly_totals[i]
    if len(x) != max_length:
        t = [x[-1]] * (max_length - len(x))
        monthly_totals[i] = np.append(monthly_totals[i], t)
    
#print monthly_totals
monthly_totals = dict(zip([str(x) for x in rates], monthly_totals))

In [ ]:
# The graphic below shows you over time where each of the rates vs points will get you
# The lower the line the better
l = Line(monthly_totals, title="Rates vs Points", ylabel='Amount Paid', xlabel='Month')
show(l)

In [ ]:
x_y = zip([np.linspace(0, 15, 15*12) for i in range(len(monthly_totals))], monthly_totals)

In [ ]:
p = figure(title="simple line example", plot_height=300, plot_width=600)
r = p.line(monthly_totals, line_width=3)

In [ ]:
'''
def update(f, w=1, A=1, phi=0):
    if   f == "sin": func = np.sin
    elif f == "cos": func = np.cos
    elif f == "tan": func = np.tan
    r.data_source.data['y'] = A * func(w * x + phi)
    push_notebook()
'''

In [ ]:
show(p, notebook_handle=True)

In [ ]:
#interact(update, f=["sin", "cos", "tan"], w=(0,100), A=(1,5), phi=(0, 20, 0.1))